In [1]:
!pip install catboost xgboost pyswarm --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 MB 3.4 MB/s eta 0:00:00


In [2]:
# === Imports ===
import os, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from pyswarm import pso

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_2017_2019 = pd.read_csv("/content/drive/MyDrive/Network Maintenace - With & without maintenace/encoded-data/No-main-encoded-dataset-2017-2019/df_encoded_no-main_2017_2019.csv")

In [4]:
df_2017_2019.shape

(6444, 23)

In [5]:
df_2017_2019.head(10)

AADT_mean_x  AADT_Single_Unit_mean_x  AADT_Combination_mean_x  \
0        40961                    820.0                   2295.0   
1        26595                    878.0                    186.0   
2        61960                    744.0                   2851.0   
3        52563                   1052.0                   2997.0   
4        13206                    211.0                   1267.0   
5        16978                    238.0                   1443.0   
6        21647                    455.0                   5196.0   
7       100449                   2008.0                   5725.0   
8        11524                    300.0                    426.0   
9         2582                     46.0                     42.0   

   Future_AADT_mean_x  IRI_mean_x  Thickness_Rigid_mean_x  \
0             40924.0        64.0                    11.5   
1             37230.0       172.8                     0.0   
2             62225.0       102.8                     9.0   
3             52581.0        65.5                    11.0   
4             13222.0        55.0                     0.0   
5             16999.0        53.3                     0.0   
6             21703.0        58.0                    12.0   
7            100481.0        67.0                    10.0   
8             12624.0       239.0                     0.0   
9              2583.0       178.3                     0.0   

   Thickness_Flexible_mean_x  Base_Thickness_mean_x  F_System_mode  \
0                        0.0                   15.0              1   
1                        3.0                    6.0              3   
2                        0.0                   18.0              2   
3                        0.0                   16.0              1   
4                        5.0                   24.0              1   
5                        5.0                   24.0              1   
6                        0.0                   18.0              1   
7                        0.0                   26.0              2   
8                        6.0                    6.0              5   
9                        3.0                    6.0              5   

   Surface_Type_mode  ...  Faulting_mean_x  IRI_mean_y  RHU_AV_x  FRZ_IDX_x  \
0                  3  ...              0.0        71.5      76.2      535.4   
1                  2  ...              0.0       198.8      67.1        3.7   
2                  3  ...              0.1       123.0      76.2      519.1   
3                  3  ...              0.0        75.0      76.4      504.3   
4                  2  ...              0.0        68.0      78.5      769.2   
5                  2  ...              0.0        55.0      78.5      769.2   
6                  3  ...              0.0        61.0      78.6      272.6   
7                  3  ...              0.0        70.5      76.3      457.0   
8                  2  ...              0.0       250.0      78.8      295.2   
9                  2  ...              0.0       184.0      78.8      295.2   

   TEMP_AVG_x  PRECIPITATION_x  Age_x  Urban_Type_rural  \
0         8.5           1068.6      5               0.0   
1        19.6            933.1     16               0.0   
2         8.7           1083.9      6               0.0   
3         8.3            963.7      4               0.0   
4         6.5           1021.7      7               1.0   
5         6.5           1021.7      7               1.0   
6         9.4           1097.5      8               0.0   
7         9.1           1050.3      7               0.0   
8         9.0           1060.7      1               0.0   
9         9.0           1060.7      9               0.0   

   Urban_Type_small urban  Urban_Type_urban  
0                     0.0               1.0  
1                     0.0               1.0  
2                     0.0               1.0  
3                     0.0               1.0  
4                     0.0               0.0  
5              

In [6]:
df_2017_2019.isnull().sum()

AADT_mean_x                   0
AADT_Single_Unit_mean_x       0
AADT_Combination_mean_x       0
Future_AADT_mean_x            0
IRI_mean_x                    0
Thickness_Rigid_mean_x        0
Thickness_Flexible_mean_x     0
Base_Thickness_mean_x         0
F_System_mode                 0
Surface_Type_mode             0
Base_Type_mode_x              0
Rutting_mean_x                0
Cracking_Percent_mean_x       0
Faulting_mean_x               0
IRI_mean_y                    0
RHU_AV_x                     12
FRZ_IDX_x                    12
TEMP_AVG_x                   12
PRECIPITATION_x              12
Age_x                         0
Urban_Type_rural              0
Urban_Type_small urban        0
Urban_Type_urban              0
dtype: int64

In [7]:
df = df_2017_2019.dropna()

In [8]:
df.shape

(6432, 23)

In [9]:
df.columns

Index(['AADT_mean_x', 'AADT_Single_Unit_mean_x', 'AADT_Combination_mean_x',
       'Future_AADT_mean_x', 'IRI_mean_x', 'Thickness_Rigid_mean_x',
       'Thickness_Flexible_mean_x', 'Base_Thickness_mean_x', 'F_System_mode',
       'Surface_Type_mode', 'Base_Type_mode_x', 'Rutting_mean_x',
       'Cracking_Percent_mean_x', 'Faulting_mean_x', 'IRI_mean_y', 'RHU_AV_x',
       'FRZ_IDX_x', 'TEMP_AVG_x', 'PRECIPITATION_x', 'Age_x',
       'Urban_Type_rural', 'Urban_Type_small urban', 'Urban_Type_urban'],
      dtype='object')

In [10]:
df.isnull().sum()

AADT_mean_x                  0
AADT_Single_Unit_mean_x      0
AADT_Combination_mean_x      0
Future_AADT_mean_x           0
IRI_mean_x                   0
Thickness_Rigid_mean_x       0
Thickness_Flexible_mean_x    0
Base_Thickness_mean_x        0
F_System_mode                0
Surface_Type_mode            0
Base_Type_mode_x             0
Rutting_mean_x               0
Cracking_Percent_mean_x      0
Faulting_mean_x              0
IRI_mean_y                   0
RHU_AV_x                     0
FRZ_IDX_x                    0
TEMP_AVG_x                   0
PRECIPITATION_x              0
Age_x                        0
Urban_Type_rural             0
Urban_Type_small urban       0
Urban_Type_urban             0
dtype: int64

In [11]:
df.columns

Index(['AADT_mean_x', 'AADT_Single_Unit_mean_x', 'AADT_Combination_mean_x',
       'Future_AADT_mean_x', 'IRI_mean_x', 'Thickness_Rigid_mean_x',
       'Thickness_Flexible_mean_x', 'Base_Thickness_mean_x', 'F_System_mode',
       'Surface_Type_mode', 'Base_Type_mode_x', 'Rutting_mean_x',
       'Cracking_Percent_mean_x', 'Faulting_mean_x', 'IRI_mean_y', 'RHU_AV_x',
       'FRZ_IDX_x', 'TEMP_AVG_x', 'PRECIPITATION_x', 'Age_x',
       'Urban_Type_rural', 'Urban_Type_small urban', 'Urban_Type_urban'],
      dtype='object')

In [12]:
num_dupes = df.duplicated(keep=False).sum()
print(f"Duplicate rows in df (all columns considered): {num_dupes}")

Duplicate rows in df (all columns considered): 2


In [13]:
# --- Remove duplicate rows across all columns ---
initial_rows = len(df)

df.drop_duplicates(inplace=True)   # modifies df in place

print(f"Duplicates removed: {initial_rows - len(df)}")
print(f"New DataFrame shape: {df.shape}")


Duplicates removed: 1
New DataFrame shape: (6431, 23)


/tmp/ipython-input-3110374706.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)   # modifies df in place


In [14]:
df.shape

(6431, 23)

In [15]:
X = df.drop(columns=['IRI_mean_y'])  # Drop target column to keep 21 input features
y = df[['IRI_mean_y']]               # Target stays (N, 1)

In [16]:
y.shape

(6431, 1)

In [17]:
print(X.shape)  # Should be (4060, 21)
print(y.shape)  # Should be (4060, 1)

(6431, 22)
(6431, 1)


In [18]:
type(y)

pandas.core.frame.DataFrame

In [19]:
# Train/val/test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [20]:
print(X_test.shape)

(965, 22)


In [21]:
print(X_train.shape)  # Should be [num_samples, 26]
print(y_train.shape)  # Should be [num_samples, 1] or [num_samples]

(4501, 22)
(4501, 1)


In [22]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
X_val_scaled = scaler_x.transform(X_val)
X_test_scaled = scaler_x.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

In [23]:
# === Torch Dataloaders ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(TensorDataset(torch.tensor(X_train_scaled, dtype=torch.float32),
                                        torch.tensor(y_train_scaled, dtype=torch.float32)),
                          batch_size=32, shuffle=True)
X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
y_val_t = torch.tensor(y_val_scaled, dtype=torch.float32).to(device)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_true_test = y_test

## ANN Optimization

In [24]:
# ==== SETUP ====
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score
from pyswarm import pso

save_dir = "/content/drive/MyDrive/Network Maintenace - With & without maintenace/outputs/outputs-no-maintain-2017-2019/pso-ANN-2017-2019-(Maintenance)-Minmax-scaler"
os.makedirs(save_dir, exist_ok=True)  # Create save directory if it doesn't exist
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

# ==== REPRODUCIBILITY ====
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==== INPUT DIMENSION ====
input_dim = X.shape[1]  # Number of features for ANN input

# ==== ACTIVATION MAP ====
activation_map = {
    "sigmoid": nn.Sigmoid(),
    "logsig": nn.LogSigmoid(),
    "tansig": nn.Tanh(),
    "relu": nn.ReLU(),
    "linear": nn.Identity()
}

# ==== ANN Model Class ====
class ANNModel(nn.Module):
    def __init__(self, input_dim, neurons, layers, activation_name):
        super().__init__()
        act_fn = activation_map.get(activation_name, nn.Tanh())  # Default Tanh if not found
        dims = [neurons]
        if layers >= 2: dims.append(max(4, neurons // 2))
        if layers >= 3: dims.append(max(3, neurons // 3))
        if layers >= 4: dims.append(max(3, neurons // 4))
        if layers == 5: dims.append(max(3, neurons // 5))

        layers_list = [nn.Linear(input_dim, dims[0]), act_fn]
        for i in range(1, len(dims)):
            layers_list.append(nn.Linear(dims[i - 1], dims[i]))
            layers_list.append(act_fn)
        layers_list.append(nn.Linear(dims[-1], 1))
        self.net = nn.Sequential(*layers_list)

    def forward(self, x):
        return self.net(x)

# ==== Metrics Function ====
def calculate_metrics(y_true, y_pred):
    n = len(y_true)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)
    mae = np.mean(np.abs(y_true - y_pred))
    epsilon = 1e-10
    mape = (100 / n) * np.sum(np.abs((y_true - y_pred) / (y_true + epsilon)))
    vaf = (1 - np.var(y_true - y_pred) / np.var(y_true)) * 100
    rsr = rmse / np.sqrt(np.mean((y_true - np.mean(y_true))**2))
    wmap = (np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))) * 100
    m20 = np.sum(np.abs(y_true - y_pred) <= 0.2 * np.abs(y_true))
    a20index = m20 / n
    return rmse, r2, mae, mape, vaf, rsr, wmap, a20index

# ==== PSO Fitness Function ====
top_results = []
activation_choices = ["sigmoid", "logsig", "tanh", "relu", "linear"]

def fitness(params):
    neurons = int(params[0])
    layers = int(round(params[1]))
    activation_idx = int(round(params[2]))
    activation_name = activation_choices[activation_idx % len(activation_choices)]

    model = ANNModel(input_dim, neurons, layers, activation_name).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_val_r2 = -np.inf
    patience = 10
    no_improve_count = 0

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

        # Validation check
        model.eval()
        with torch.no_grad():
            y_pred_val = model(X_val_t).cpu().numpy()
            y_true_val = y_val_t.cpu().numpy()
        _, r2, *_ = calculate_metrics(y_true_val, y_pred_val)

        if r2 > best_val_r2:
            best_val_r2 = r2
            no_improve_count = 0
        else:
            no_improve_count += 1

        if no_improve_count >= patience:
            break

    # Final metrics
    rmse, r2, mae, mape, vaf, rsr, wmap, a20index = calculate_metrics(y_true_val, y_pred_val)

    top_results.append((r2, rmse, mae, mape, vaf, rsr, wmap, a20index, neurons, layers, activation_name))
    top_results.sort(key=lambda x: x[0], reverse=True)
    if len(top_results) > 3:
        top_results.pop()
    return -r2

# ==== Run PSO ====
top_results.clear()
lb = [
    max(8, int(0.3 * input_dim)),     # neurons lower bound
    1,                                # layers lower bound
    0                                 # activation index lower bound
]

ub = [
    min(512, int(2 * input_dim)),      # neurons upper bound
    10,                               # layers upper bound
    len(activation_choices) - 1        # activation index upper bound
]

best_params, _ = pso(fitness, lb, ub, swarmsize=10, maxiter=5)

best_neurons = int(best_params[0])
best_layers = int(round(best_params[1]))
activation_idx = int(round(best_params[2]))
best_activation = activation_choices[activation_idx]

# ==== Print Top 3 PSO Results ====
print("\n🔎 Top 3 neuron, layer, activation Configurations:")
for i, (r2, rmse, mae, mape, vaf, rsr, wmap, a20index, n, l, a) in enumerate(top_results):
    print(
        f"{i+1}. R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, "
        f"MAPE: {mape:.4f}%, VAF: {vaf:.4f}%, RSR: {rsr:.4f}, "
        f"WMAP: {wmap:.4f}%, a20index: {a20index:.4f}, "
        f"Neurons: {n}, Layers: {l}, Activation: {a}"
    )

# ==== Grid Search over Optimizer and LR ====
learning_rates = [0.01, 0.001, 0.0005]
optimizers = ['adam', 'sgd', 'rmsprop']
results = []

for lr in learning_rates:
    for opt_name in optimizers:
        model = ANNModel(input_dim, best_neurons, best_layers, best_activation).to(device)
        criterion = nn.MSELoss()
        if opt_name == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=lr)
        elif opt_name == 'sgd':
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        elif opt_name == 'rmsprop':
            optimizer = optim.RMSprop(model.parameters(), lr=lr)

        for epoch in range(300):
            model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                loss = criterion(model(xb), yb)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred_test = model(X_test_t).cpu().numpy()
            y_pred_test = scaler_y.inverse_transform(y_pred_test)
        rmse, r2, mae, mape, vaf, rsr, wmap, a20index = calculate_metrics(y_true_test, y_pred_test)

        # ✅ Convert everything to plain floats before appending
        results.append((opt_name, lr, float(r2), float(rmse), float(mae), float(mape),
                        float(vaf), float(rsr), float(wmap), float(a20index)))

results = sorted(results, key=lambda x: x[2], reverse=True)

# ==== Print Top 3 Optimizer + LR Combinations ====
print("\n⚙️ Top 3 Optimizer + LR Combinations (All Metrics):")
for i, (opt, lr, r2, rmse, mae, mape, vaf, rsr, wmap, a20index) in enumerate(results[:3]):
    print(
        f"{i+1}. Optimizer: {opt}, LR: {lr}, "
        f"R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, "
        f"MAPE: {mape:.4f}%, VAF: {vaf:.4f}%, "
        f"RSR: {rsr:.4f}, WMAP: {wmap:.4f}%, "
        f"a20index: {a20index:.4f}"
    )

# ==== 🔍 Best Overall Combination ====
best_result = results[0]
best_optimizer, best_lr, best_r2, best_rmse, best_mae, best_mape, best_vaf, best_rsr, best_wmap, best_a20index = best_result

print("\n🏆 Best Overall ANN Model Configuration:")
print(f"🔸 Neurons: {best_neurons}")
print(f"🔸 Hidden Layers: {best_layers}")
print(f"🔸 Activation Function: {best_activation}")
print(f"🔸 Optimizer: {best_optimizer}")
print(f"🔸 Learning Rate: {best_lr}")
print(f"🔸 R² Score: {best_r2:.4f}")
print(f"🔸 RMSE: {best_rmse:.4f}")
print(f"🔸 MAE: {best_mae:.4f}")
print(f"🔸 MAPE: {best_mape:.4f}%")
print(f"🔸 VAF: {best_vaf:.4f}%")
print(f"🔸 RSR: {best_rsr:.4f}")
print(f"🔸 WMAP: {best_wmap:.4f}%")
print(f"🔸 a20index: {best_a20index:.4f}")


Stopping search: maximum iterations reached --> 5

🔎 Top 3 neuron, layer, activation Configurations:
1. R²: 0.8913, RMSE: 0.0528, MAE: 0.0312, MAPE: 4794506.0000%, VAF: 89.1530%, RSR: 0.3297, WMAP: 14.5058%, a20index: 0.7036, Neurons: 35, Layers: 1, Activation: relu
2. R²: 0.8885, RMSE: 0.0535, MAE: 0.0331, MAPE: 6806269.0000%, VAF: 89.0568%, RSR: 0.3340, WMAP: 15.4246%, a20index: 0.6435, Neurons: 31, Layers: 1, Activation: relu
3. R²: 0.8883, RMSE: 0.0535, MAE: 0.0314, MAPE: 8199503.5000%, VAF: 88.9048%, RSR: 0.3342, WMAP: 14.6386%, a20index: 0.6808, Neurons: 37, Layers: 3, Activation: relu


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:4006: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/tmp/ipython-input-1749425244.py:188: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  results.append((opt_name, lr, float(r2), float(rmse), float(mae), float(mape),
/tmp/ipython-input-1749425244.py:189: FutureWarning: C


⚙️ Top 3 Optimizer + LR Combinations (All Metrics):
1. Optimizer: sgd, LR: 0.01, R²: 0.9142, RMSE: 16.8628, MAE: 9.8713, MAPE: 8.5401%, VAF: 91.4347%, RSR: 0.2929, WMAP: 8.9428%, a20index: 0.9316
2. Optimizer: rmsprop, LR: 0.0005, R²: 0.9095, RMSE: 17.3208, MAE: 12.0710, MAPE: 11.5157%, VAF: 91.4483%, RSR: 0.3008, WMAP: 10.9356%, a20index: 0.8611
3. Optimizer: sgd, LR: 0.001, R²: 0.9083, RMSE: 17.4360, MAE: 10.7033, MAPE: 9.6274%, VAF: 90.8632%, RSR: 0.3028, WMAP: 9.6966%, a20index: 0.9067

🏆 Best Overall ANN Model Configuration:
🔸 Neurons: 35
🔸 Hidden Layers: 1
🔸 Activation Function: relu
🔸 Optimizer: sgd
🔸 Learning Rate: 0.01
🔸 R² Score: 0.9142
🔸 RMSE: 16.8628
🔸 MAE: 9.8713
🔸 MAPE: 8.5401%
🔸 VAF: 91.4347%
🔸 RSR: 0.2929
🔸 WMAP: 8.9428%
🔸 a20index: 0.9316


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:4006: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/tmp/ipython-input-1749425244.py:188: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  results.append((opt_name, lr, float(r2), float(rmse), float(mae), float(mape),
/tmp/ipython-input-1749425244.py:189: FutureWarning: C

## FINAL ANN PARAMETER OPTIMISATION

In [25]:
# ==== Final ANN Training with Best Hyperparameters ====
final_model = ANNModel(input_dim, best_neurons, best_layers, best_activation).to(device)
criterion = nn.MSELoss()

# Select optimizer
if best_optimizer == 'adam':
    optimizer = optim.Adam(final_model.parameters(), lr=best_lr)
elif best_optimizer == 'sgd':
    optimizer = optim.SGD(final_model.parameters(), lr=best_lr, momentum=0.9)
elif best_optimizer == 'rmsprop':
    optimizer = optim.RMSprop(final_model.parameters(), lr=best_lr)

# ==== Train the ANN ====
for epoch in range(300):
    final_model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(final_model(xb), yb)
        loss.backward()
        optimizer.step()

# ==== Predict ====
final_model.eval()
with torch.no_grad():
    y_pred_test_ann = final_model(X_test_t).cpu().numpy()
    y_pred_test_ann = scaler_y.inverse_transform(y_pred_test_ann)

# ==== Calculate ANN Metrics ====
rmse_ann, r2_ann, mae_ann, mape_ann, vaf_ann, rsr_ann, wmap_ann, a20index_ann = calculate_metrics(
    y_true_test, y_pred_test_ann
)

# ==== Convert Series to scalar if needed ====
rmse_ann = float(rmse_ann)
r2_ann = float(r2_ann)
mae_ann = float(mae_ann)
mape_ann = float(mape_ann)
vaf_ann = float(vaf_ann)
rsr_ann = float(rsr_ann)
wmap_ann = float(wmap_ann)
a20index_ann = float(a20index_ann)

# Save ANN metrics as a tuple
ann_metrics = (
    rmse_ann,
    r2_ann,
    mae_ann,
    mape_ann,
    vaf_ann,
    rsr_ann,
    wmap_ann,
    a20index_ann
)

# ==== Print final ANN metrics ====
print("\n📊 Final ANN Performance Metrics:")
print(f"🔹 R² Score: {r2_ann:.4f}")
print(f"🔹 RMSE: {rmse_ann:.4f}")
print(f"🔹 MAE: {mae_ann:.4f}")
print(f"🔹 MAPE: {mape_ann:.4f}%")
print(f"🔹 VAF: {vaf_ann:.4f}%")
print(f"🔹 RSR: {rsr_ann:.4f}")
print(f"🔹 WMAP: {wmap_ann:.4f}%")
print(f"🔹 a20index: {a20index_ann:.4f}")



📊 Final ANN Performance Metrics:
🔹 R² Score: 0.9070
🔹 RMSE: 17.5589
🔹 MAE: 11.6118
🔹 MAPE: 10.8957%
🔹 VAF: 90.9898%
🔹 RSR: 0.3050
🔹 WMAP: 10.5196%
🔹 a20index: 0.8912


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:4006: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/tmp/ipython-input-1533803780.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  r2_ann = float(r2_ann)
/tmp/ipython-input-1533803780.py:38: FutureWarning: Calling float on a single element Series is deprecated and 